In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
rusdr_meta = pd.read_excel("rusdracor-metadata.xlsx")
rusdr_meta.head()

## Работающий код, но ~7 пьес не считываются нормально.


In [ ]:
def extract_speeches_from_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
    speeches_dict = {}
    acts = root.findall('.//tei:div[@type="act"]', namespaces)
    if not acts:
        acts = [root.find('.//tei:body', namespaces)]
    for act_index, act in enumerate(acts, start=1):
        act_dict = {}
        for sp in act.findall('.//tei:sp', namespaces):
            speaker = sp.find('tei:speaker', namespaces)
            if speaker is not None:
                speaker_name = speaker.text.strip()
                lines = []
                for p in sp.findall('tei:p', namespaces):
                    line = ''.join(p.itertext()).strip()
                    lines.append(line)
                l_texts = []
                for l in sp.findall('tei:l', namespaces):
                    l_text = ''.join(l.itertext()).strip()
                    l_texts.append(l_text)
                if l_texts:
                    lines.append(' '.join(l_texts))
                act_dict.setdefault(speaker_name, []).extend(lines)
        speeches_dict[act_index] = act_dict
    return speeches_dict

### Для этих 7 пьес

In [ ]:
def extract_speeches_from_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
    speeches_dict = {}
    acts = root.findall('.//tei:div[@type="act"]', namespaces)
    if not acts:
        acts = [root.find('.//tei:body', namespaces)]
    for act_index, act in enumerate(acts, start=1):
        act_dict = {}
        for sp in act.findall('.//tei:sp', namespaces):
            speaker = sp.find('tei:speaker', namespaces)
            if speaker is not None:
                speaker_name = speaker.text.strip()
                p_texts = []
                for p in sp.findall('.//tei:p', namespaces):
                    line = ''.join(p.itertext()).strip()
                    p_texts.append(line)
                lg_texts = []
                for lg in sp.findall('.//tei:lg', namespaces):
                    l_texts = []
                    for l in lg.findall('tei:l', namespaces):
                        l_text = ''.join(l.itertext()).strip()
                        l_texts.append(l_text)
                    if l_texts:
                        lg_texts.append(' '.join(l_texts))
                act_dict.setdefault(speaker_name, []).extend(p_texts + lg_texts)
        speeches_dict[act_index] = act_dict
    return speeches_dict

## Объединяем

In [ ]:
corpus_dir = 'corpus'

speech_data = []

for _, row in rusdr_meta.iterrows():
    file_name = row['name']
    file_path = os.path.join(corpus_dir, file_name)
    speeches_dict = extract_speeches_from_xml(file_path + ".xml")
    speech_row = {
        'title': row['title'],
        'author': row['firstAuthor'],
        'year': row['yearNormalized'],
        'genre': row['normalizedGenre']
    }
    for act_number, act_dict in speeches_dict.items():
        speech_row[str(act_number)] = act_dict
    speech_data.append(speech_row)
speech_df = pd.DataFrame(speech_data)